In [28]:
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Merge, LSTM, Dense
import numpy as np

face_dim = 126
eyes_dim = 20

data_dim = face_dim + eyes_dim
hidden_state_dim = int(data_dim / 4)
timesteps = 2
nb_classes = 6

def fork (model, n=2):
    forks = []
    for i in range(n):
        f = Sequential()
        f.add (model)
        forks.append(f)
    return forks

In [29]:
for step in range(timesteps):
    encoder_a = Sequential()
    encoder_a.add(LSTM(hidden_state_dim, input_shape=(1, data_dim), return_sequences=True))

    encoder_b = Sequential()
    encoder_b.add(LSTM(hidden_state_dim, input_shape=(1, data_dim), return_sequences=True))

#     encoder_c = Sequential()
#     encoder_c.add(LSTM(hidden_state_dim, input_shape=(1, data_dim)))

#     encoder_d = Sequential()
#     encoder_d.add(LSTM(hidden_state_dim, input_shape=(1, data_dim)))

    merge = Sequential()
    merge.add(Merge([encoder_a, encoder_b], mode='concat'))
    
    encoder_a, encoder_b = fork(merge)
    encoder_a.add(LSTM(hidden_state_dim, input_shape=(1, data_dim), return_sequences=True))
    encoder_b.add(LSTM(hidden_state_dim, input_shape=(1, data_dim), return_sequences=True))

model = Sequential()
model.add(Merge([encoder_a, encoder_b], mode='concat'))

model.add(Dense(32, activation='relu'))
model.add(Dense(nb_classes, activation='softmax'))

sgd = SGD(lr=0.1, decay=1e-5, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)



c:\users\amvx\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:15: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  from ipykernel import kernelapp as app
c:\users\amvx\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:22: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [30]:
# generate dummy training data
x_train_a = np.random.random((1000, timesteps, face_dim))
x_train_b = np.random.random((1000, timesteps, eyes_dim))
y_train = np.random.random((1000, nb_classes))
concat_train = np.concatenate([x_train_a, x_train_b], axis=2)

# generate dummy validation data
x_val_a = np.random.random((100, timesteps, face_dim))
x_val_b = np.random.random((100, timesteps, eyes_dim))
y_val = np.random.random((100, nb_classes))
concat_test = np.concatenate([x_val_a, x_val_b], axis=2)

model.fit([concat_train, concat_train], y_train,
            batch_size=64, nb_epoch=5,
            validation_data=([concat_test, concat_test], y_val))

c:\users\amvx\appdata\local\programs\python\python35\lib\site-packages\keras\models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


ValueError: Error when checking input: expected lstm_169_input to have shape (None, 1, 146) but got array with shape (1000, 2, 146)